# Prediction of the isotopic inventory in a nuclear reactor core

*Benjamin Dechenaux, Jean-Baptiste Clavel, Cécilia Damon (IRSN)*

## Introduction 

Matter contained inside a nuclear reactor undergoes irradiation that causes successive cascades of nuclear reactions, modifying its atomic composition. Knowledge of this time evolving composition is an important parameter used to model the behavior of a nuclear reactor but is also a crucial input for safety studies associated with its operation and is a key input for the mitigation of severe accident for it constitute, for instance the source term of the release of radioactive isotopes in the environment. 

The modelling of the change in the atomic composition of irradiated materials over time is usually achieved using time expensive Monte Carlo simulation of the system being studied. Although precise, this calculation scheme has been found inadapted in crisis (i.e. accidental) situations, where faster calculation schemes have to be developped. 

This project aims at constructing a surrogate model capable at predicting the time changing nuclear inventory of a typical reactor of the French fleet. 

## Requirements for running the notebook 

To properly run the notebook, the following Python modules must be imported :

In [ ]:
# Required for running the notebook 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import pickle
import string

# sklearn dependences are used to build a baseline model 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

# Keras dependences used to established an example of Neural Network
from keras import models
from keras import layers
from keras.optimizers import Adagrad


%matplotlib inline

## Description of the data 

The atomic composition of the material subjected to irradiation usually involves thousands of isotopes. However, most of them have short lifetimes (characteristic decay time) and are not of critical interest for the safety of the facility or the mitigation of a reactor accident. 

In this project, we propose to track the temporal evolution of __26 isotopes__ that have a significant impact on the reactor behaviour or are isotopes whose composition is a key input for the study of the mitigation of an accident (they might be isotopes that are usually found to be released in the environment in accidental scenario, or are important for safety reasons in general). 

For non-proliferation reasons, the isotopes were renamed and each of them are identified by a unique letter of the alphabet (e.g. the first isotope whose composition we would like to track is named "A").   

So we'll be interested in the evolution of the content in isotopes "A", "B", ..., "Z" inside a reactor with time. The isotope content at each time is expressed in some kind of density unity ( ~ number of atoms per cubic cm).  

### Modelling the irradiation in a nuclear reactor

To model the irradiation conditions of the nuclear material, we suppose a simplified scheme for the temporal evolution.

Matter is put inside the reactor for a grand total of __1825 days__ (i.e. 5 years). 

This total period is subdivided into 5 different irradiation __cycles__ where each cycle correspond to a 300 days period with a certain configuration of the reactor (its operating power), charaterized by a single parameter *pi* (with i running from 1 -> 5).  

Between 2 cycles, a period of 65 days was added and correspond to say, a maintenance period for the reactor, where the fuel is not irradiated (which note, doesn't mean the isotopes don't evolve). During these intercycle period, we don't track the evolution of the isotopes content.

To summarize, the nuclear fuel is put inside of a reactor for a total of 1825 days with the following history :

* The fuel is irradiated for 300 days (from T = 0 to T = 300) with parameter *p1*
* The fuel is put to rest for 65 days
* The fuel is irradiated for 300 days (from T = 365 to T = 665) with parameter *p2*
* The fuel is put to rest for 65 days
* The fuel is irradiated for 300 days (from T = 730 to T = 1030) with parameter *p3*
* The fuel is put to rest for 65 days
* The fuel is irradiated for 300 days (from T = 1095 to T = 1395) with parameter *p4*
* The fuel is put to rest for 65 days
* The fuel is irradiated for 300 days (from T = 1460 to T = 1760) with parameter *p5*
* The fuel is put to rest for 65 days


For the sake of the exercise, the composition of each isotope will be tracked on a __20 days__ time interval basis, except for the periods where the fuel is put to rest, where the interval is 65 days (i.e. the evolution of fuel composition is not being tracked, which doesn't mean it doesn't evolve...) . 


So in the end, for a given set of input data (which consist of the inital composition of isotopes A -> H and the 5 parameters p1,...,p5), the result is a time series of length 81 (initial composition + 80 timesteps).


### Description of the available data

The database we built is composed of a total of __1120__ different reactor histories (i.e. time series), that we generated varying the initial conditions (i.e. the input parameters, see below) of the system and performing a detailled simulation of the reactor evolution in each case.  

For each of the 1120 simulations, we kept track of the composition of the 26 isotopes we are interested in. These are the output data of the database (see below). 


#### Input data 

The input data is composed of the initial composition of the matter before irradiation and the 5 parameters p1, .. p5 that specify the global operating conditions of the reactor for each irradiation cycle.

**This initial composition of the matter is only composed of isotopes __A to H__ ** (all the other are always equal to zero at the initial time T=0).

For each of the generated data point, the initial conditions (which is the input for any surrogate model) is composed of those 13 paramaters :

* Initial composition in terms of isotopes A --> H (8 parameters)
* Operating conditions of the reactor p1, ... , p5 (5 parameters)

For each of the 1120 differents simulations that were done, a unique set of values was picked in this initial input space using the latin hypercube method.   

#### Output data 

For each point in the input space, a simulation of the time evolving composition of matter was performed. The result of a simulation is a time series of length 81 (initial composition + 80 timesteps) for each isotope (form A to Z. Note that isotopes I -> Z always have zero composition at T=0). 

Those times series are stored in CSV output files that can be found under the _data/train_ and _data/test_ folders.

__Remark__

To ease the manipulation of the time series, the additional p1, ..., p5 input parameters were added to each of their corresponding timesteps (e.g. a time series named p1 which have the same value for each entry of the time series).


### Loading the data 

The 1120 simulations have been split into two *training* and *testing* datasets.

* The *training* dataset is composed of 920 simulations and are accessible in CSV format undert the __train__ folder
* The *testing* dataset is composed of 200 simulations and are accessible in CSV format undert the __test__ folder

To ease the reading of both the training and testing datasets, serialized *pandas* dataframes have been pre-prepared and can be loaded using *pickle*

In [ ]:
dtrain = pickle.load( open("data/train_data_python3.pickle", "rb") )
dtest = pickle.load( open("data/test_data_python3.pickle", "rb") )

In these dataframes, data have been concatenated one on top of the other.


In [ ]:
dtrain.head(5)

To separate from one file to the other, use the index of the dataframes or the "times" column.

The input data for instance is composed of the value of each T=0 entry in the datasets. To obtain those values, simply select every entry having T=0 in the train or test datasets : 

In [ ]:
dtrain.loc[0].shape  # equivalent to dtrain.loc[dtrain["times"] == 0.]

As said before, the train dataset regroup a total of 920 simulations that were performed varying the 13 input parameters listed above. Here the dataset is found to have 32 parameters at T=0, but only 13 are non zero, as can be seen using :  

In [ ]:
dtrain.iloc[0]

As adverstised, the initial compositions for isotopes "I" -> "Z" are zero, leaving only 13 input parameters. 
At T= 1825 days the (81th timestep), the composition has evolved :

In [ ]:
dtrain.iloc[[0,80]]

To get all of the timesteps, one can use the barbaric but efficient :

In [ ]:
timesteps = sorted(list(set(dtrain["times"])))
print(timesteps)

## Remarks concerning the input space

The input space is made of the T=0 entry of each of the 1120 time series. 
It is composed of the initial composition of isotopes A to H and the 5 parameters p1 --> p5. 

Let us focus on the initial compositions of isotopes A --> H :

In [ ]:
alphabet = list(string.ascii_uppercase) #  to ease the manipulation of the data

# The input compositions are isotopes A -> H  
input_compos= alphabet[0:8]

# The input paramters are composed of the input_compos + parameters p1 to p5 
input_params = input_compos + ["p1", "p2", "p3", "p4", "p5"]

# concatenate the T=0 entries for both the train and test datasets
input_space = pd.concat([dtrain.loc[0][input_params] , dtest.loc[0][input_params]  ]) 

In [ ]:
input_space.shape

In [ ]:
# Plot the initial compositions on a pairplot
sns.set_style("dark")
sns.set(rc={'figure.figsize':(16,20)})
sns.pairplot( input_space[input_compos] )
plt.show()

One can observe a non-uniform covering of the input space for the training dataset. and sometime clear cut in this input space sampling. 

This is because ratio of different quantities where used to determine boundaries in the sampling of the input parameter space. This was based on physical consideration and it was realised afterwards that it maybe wasn't the cleverest way to sample the data...  

## Normalization of the data

It is important to note that the composition data that make up the database are very heterogeneous. Indeed, the isotopes present in this database have typical compositions that can present orders of magnitude of differences . This can pose serious problems of standardization to succeed in learning the data at best. 

Let us for instance plot the distributions of the isotopes that makes up the input parameter space (i.e. isotopes A to H) at the initial T=0 and final T=1825 times : 

In [ ]:
temp = pd.DataFrame()
for i in input_compos:
    temp=temp.append( pd.DataFrame( {"composition (a.u.)":dtrain[i].loc[0] , "isotope":[i for j in dtrain[i].loc[0]] , "time":["initial" for j in dtrain[i].loc[0]]} ) )
    temp=temp.append( pd.DataFrame( {"composition (a.u.)":dtrain[i].loc[80] , "isotope":[i for j in dtrain[i].loc[80]] , "time":["final" for j in dtrain[i].loc[80]]} ) )

## plot a violin plot for both the initial and final compositions of the input_compos  
sns.set(rc={'figure.figsize':(14,8)})
sns.violinplot(data=temp,x="isotope",y="composition (a.u.)",hue="time",split=True, inner="quartile", linewidth=1
               , palette={"initial":"cornflowerblue","final":".85"})
sns.despine(left=True)


As can be seen from the figure, most matter inside the reactor is made up of isotope B. 

The difficulty lies in the fact that we want, in the end, to have a precise __relative__ error on each individual isotope because it can happen that an isotope which is present in small quantities still has a non-negligible impact on the safety of the reactor and on the mitigation of an accident.

So we can't just be good in the absolute overall composition of the matter : we need to be sufficiently precise for each individual isotope !



The differences between the isotopes for the final compositions, adding all of the 26 isotopes, is even more flagrant :  

In [ ]:
temp = pd.DataFrame()

for i in alphabet:
    temp=temp.append( pd.DataFrame( {"composition (a.u.)":dtrain[i].loc[80] , 
                                     "isotope":[i for j in dtrain[i].loc[80]] , 
                                     "time":["final" for j in dtrain[i].loc[80]]} ) )
  
sns.set(rc={'figure.figsize':(16,8)})
sns.violinplot(data=temp,x="isotope",y="composition (a.u.)", inner="box")
sns.despine(left=True)

There, we want to be sufficiently precise on each individual isotopes, even if there exist ~ up to 5 orders of magnitudes between some isotopes ! 

In [ ]:
dtrain.B.max(), dtrain.Z.max()

A clever normalization of the input and output data is of paramount importance. 
In this notebook, we will simply use the most straightforward normalization possible : we will divide each composition by the maximum found for each isotope :

In [ ]:
max_train_data = dtrain.max()
dtrain_norm = dtrain/max_train_data


max_test_data = dtest.max()
dtest_norm = dtest/max_test_data

It makes things a bit better 

In [ ]:
temp = pd.DataFrame()

for i in alphabet:
    temp=temp.append( pd.DataFrame( {"composition (a.u.)":dtrain_norm[i].loc[80] , 
                                     "isotope":[i for j in dtrain_norm[i].loc[80]] , 
                                     "time":["final" for j in dtrain_norm[i].loc[80]]} ) )
  
sns.set_style("dark")
sns.set(rc={'figure.figsize':(16,8)})
sns.violinplot(data=temp,x="isotope",y="composition (a.u.)", inner="box")
sns.despine(left=True)

It partially solves the problem, except for isotope Z which remains small as compared to the others...

## A simple baseline algorithm  

To benchmark the performances of a machine learning algorithm, we first try to build a simple baseline method.

Our goal is to predict the composition of matter inside the reactor **at any given time** by just using its initial composition (isotopes A --> H) and the parameters p1, ..., p5.

The baseline algorithm presented here simply performs a linear regression for each and every time step and for each isotope. 

Let us first reshape the data into a form that will be more usefull for the rest of this notebook 

In [ ]:
train_data = pd.DataFrame()

for simu in range(0,920): # loop over each and every simulations (the 920 train data points)
    a = dtrain_norm.iloc[simu*81:(simu+1)*81]  # slice training dataset (the normalised dataset)
    b = a[input_params].iloc[0].to_dict()      # get T=0 data (the input data)
    for i in range(1,len(timesteps) ):         # loop over timesteps (from T=20 days to T=1825 days)
        c = a[alphabet + ["times"]]            # get target data ( compos A --> Z at current timestep + time)
        c.columns = ["target_"+j for j in alphabet] + ["times"]  
        c = c.iloc[i].to_dict()
        
        b.update(c)                            # merge dictionaries having input and output data 
        
        train_data = train_data.append( b , ignore_index=True)  # append current data (input + ouput) into a new dataset 

In [ ]:
print(dtrain_norm.shape)
train_data_cd = dtrain_norm[alphabet].add_prefix('target_')
train_data_cd["times"] = dtrain_norm["times"]
display(train_data_cd)

temp = pd.DataFrame(np.repeat(dtrain_norm.loc[0][input_params].values, 81, axis=0), columns=input_params).reset_index(drop = True)
train_data_cd = pd.concat([temp, train_data_cd.reset_index(drop=True)], axis = 1)
display(train_data_cd)

In [ ]:
test_data = pd.DataFrame()

for simu in range(0,200): # loop over each and every simulations (the 920 train data points)
    a = dtest_norm.iloc[simu*81:(simu+1)*81]  # slice training dataset (the normalised dataset)
    b = a[input_params].iloc[0].to_dict()      # get T=0 data (the input data)
    for i in range(1,len(timesteps) ):         # loop over timesteps (from T=20 days to T=1825 days)
        c = a[alphabet + ["times"]]            # get target data ( compos A --> Z at current timestep + time)
        c.columns = ["target_"+j for j in alphabet] + ["times"]  
        c = c.iloc[i].to_dict()
        
        b.update(c)                            # merge dictionaries having input and output data 
        
        test_data = test_data.append( b , ignore_index=True)  # append current data (input + ouput) into a new dataset 


Now for each isotope and each timestep, we will try to construct a linear regression that best fit the data  

In [ ]:
res_target = pd.DataFrame()

for abc in alphabet:
    
    output = "target_"+abc
    
    train_target = train_data.groupby(input_params)[output].apply(list).apply(pd.Series)\
                .rename(columns=lambda x: abc+str(x+1)).reset_index()
    test_target = test_data.groupby(input_params)[output].apply(list).apply(pd.Series)\
                .rename(columns=lambda x: abc+str(x+1)).reset_index()

    for i in range(80):
        ytimes = abc+str(i+1)

        X_train = train_target[input_params]
        y_train = train_target[ytimes]
        X_test = test_target[input_params]
        y_test = test_target[ytimes]

        reg_ols = LinearRegression()
        y_pred_ols = reg_ols.fit(X_train, y_train).predict(X_test)
        r2_score_ols = r2_score(y_test, y_pred_ols)
        mae_score_ols = mean_absolute_error(y_test, y_pred_ols)
        
        res_target["tar_%s_%i"%(abc,i+1)] = y_test
        res_target["res_%s_%i"%(abc,i+1)] = y_pred_ols
        #print("OLS R2/MAE scores: %f/%f" %(r2_score_ols,mae_score_ols))


The results for isotope A (in blue : true data,  in red : predicted data)

In [ ]:
tar = res_target[ ["tar_A_%i"%(i) for i in range(1,81)] ]
res = res_target[ ["res_A_%i"%(i) for i in range(1,81)] ]

plt.figure(figsize=(20,10))
for i in range( 0, tar.shape[0]):
    plt.plot( range( 80)  , tar.iloc[i,0:] ,"bo"  )
    plt.plot( range( 80)  , res.iloc[i,0:] ,"ro"  )

plt.xlabel(" Timestep")
plt.ylabel(" Composition in isotope A (a.u.)")
plt.show()


Summing over all timesteps, here's the average relative error that we obtain :

In [ ]:
print( "Mean accuracy (%) :")

for abc in alphabet:

    res = []
    for i in range(1,81):
        res = res + (np.fabs(res_target["tar_%s_%i"%(abc,i)] - res_target["res_%s_%i"%(abc,i)])/res_target["tar_%s_%i"%(abc,i)] ).tolist()
    res = 100 - np.array(res)*100
    print( abc , "  %.2f"%(res.mean()) )



A machine learning algorithm must beat those numbers to pretend doing any decent job ! 

Note : isotopes R and W have negative percentage values becaus eof the substraction : "100 - np.array(res)*100"

That just mean that their prediction error is very large ( > 600 % !)

## A naive neural network attempt

We will use a standard neural network built from the keras library. 

To treat time, we choose simply to treat this parameter has an input variable that will be fed to the neural network.


Our goal is to predict the composition of matter inside the reactor **at any given time** by just using its initial composition (isotopes A --> H) and the parameters p1, ..., p5.

To makes thing a bit more simple, we'll use the transformed training and testing datasets. this will ease the use of a network with time is an input parameter.

The training dataset looks like : 

In [ ]:
train_data

And for the test dataset

In [ ]:
test_data

Build a basic neural network architecture ( 13 input parameters + time)

In [ ]:
# Build simple MLP 

model = models.Sequential()
model.add( layers.Dense(100,activation='relu', input_shape= (train_data[input_params + ["times"]].shape[1],))  )
model.add( layers.Dense(100,activation='relu'))
model.add( layers.Dense(100,activation='relu'))
model.add( layers.Dense(50,activation='relu'))
model.add( layers.Dense(26) )

model.summary()

To train the surrogate model, We used MSE and MAE as loss and validation respectively. A more refined choice might be necessary ... 

In [ ]:
model.compile( optimizer=Adagrad() , loss="mse", metrics=["mape"])  # loss : either mse or logcosh

history = model.fit( train_data[input_params + ["times"]] , train_data[["target_"+i for i in alphabet]] , 
                    epochs=500, batch_size=20, validation_split=0.3,shuffle=True,verbose=0) 


In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1 , len(loss)+1)

plt.subplot( 2 , 1, 1 )
plt.plot(epochs , loss , "bo")
plt.plot(epochs , val_loss , "r")
plt.yscale("log")
plt.subplot( 2 , 1, 2 )
plt.plot(epochs , history.history["mean_absolute_percentage_error"] , "bo")
plt.plot(epochs , history.history["val_mean_absolute_percentage_error"]  , "r")

#plt.yscale("log")
plt.show()

In [ ]:
# Use NN to predict the compos of test data 
res = model.predict( test_data[input_params + ["times"]]  )

for i,val in enumerate(alphabet):
    test_data["res_"+val] = res[:,i]

print( "Mean accuracy (%) :")

for i,name in enumerate(alphabet):
    test_data["diff_"+name] = (test_data["target_"+name]-test_data["res_"+name])/test_data["target_"+name]
    print(name, " %.2f"%(100-np.fabs(test_data["diff_"+name].mean()*100) ) )

The overall performances are almost perfect for some isotopes and very bad for others. We might want to investigate the problem in more depth :

In [ ]:
plt.plot( test_data["times"]*max_test_data.times, test_data["target_A"] ,"bo", label="True value" )
plt.plot( test_data["times"]*max_test_data.times, test_data["res_A"] ,"ro", label="Predicted value" )

plt.xlabel(" Time (days)")
plt.ylabel(" Composition in isotope A (a.u.)")
plt.legend()
plt.show()

Isotope A as a nice temporal behaviour, it is quite easy to get good results with this kind of isotope.

In [ ]:
plt.plot( test_data["times"]*max_test_data.times, test_data["target_W"] ,"bo", label="True value" )
plt.plot( test_data["times"]*max_test_data.times, test_data["res_W"] ,"ro", label="Predicted value" )

plt.xlabel(" Time (days)")
plt.ylabel(" Composition in isotope W (a.u.)")
plt.legend()
plt.show()

The overall temporal behaviour of isotope W is also globally well reproduced, except for the most extreme bahviours, where certainly more data would have helped.

In [ ]:
plt.plot( test_data["diff_B"]*100 , test_data["times"]*max_test_data['times'], "bo" )
plt.xscale('log')

plt.xlabel('Relative error')
plt.ylabel( 'time (days)' )
plt.show()

The relatives errors ( true - predicted)/true are very nice for isotope B, and stays constant with time

In [ ]:
plt.plot( test_data["diff_W"]*100 , test_data["times"]*max_test_data['times'], "bo" )
plt.xscale('log')

plt.xlabel('Relative error')
plt.ylabel( 'time (days)' )
plt.show()

For more complex isotopes, like W, the error is very large at the beggining, because the initial composition is very low. It gets better at the end of irradiation but is still too large on overall...

Let us look at the dirstibutions of the true and predicted compositions for the final time (T = 1825 days) 

In [ ]:
test_data_final = test_data[ test_data["times"]*max_test_data.times == 1825.0] 

d={}

d["Composition (a.u.)"] = []
d["Isotope"] = [ ]
d["Type"] = []


for i in alphabet: 
    
    
    d["Composition (a.u.)"] += test_data_final["target_"+i].to_list() + test_data_final["res_"+i].to_list()
    d["Isotope"] += [i]*400
    d["Type"] += ["True composition"]*200 + ["Predicted composition"]*200

temp = pd.DataFrame(d)

sns.set(rc={'figure.figsize':(16,8)})
sns.violinplot(data=temp,x="Isotope",y="Composition (a.u.)",hue="Type",
               split=True, inner="quartile", linewidth=1
               , palette={"True composition":"cornflowerblue","Predicted composition":".85"})
sns.despine(left=True)

To conclude, let us highlight some discrepancies that we believe is typical from the hard cuts that were performed in the input parameter space :

In [ ]:
plt.plot( test_data_final["D"], test_data_final["H"], "bo")
plt.plot( test_data_final[np.fabs(test_data_final["diff_H"]) > 0.25]["D"], 
         test_data_final[np.fabs(test_data_final["diff_H"]) > 0.25]["H"], "ro")

plt.show()

The points in blue on the figure are the initial compositions of the test points in the input parameter space D vs H.

In red, the same test points are highlighted only if the error of the prediction at T=1825 days is greater than 25%. All of the points with large errors are located in the corner of the distribution... 
